In [1]:
import xarray as xr
import pandas as pd
import numpy as np
from datetime import datetime

## Things we want to "demo" in this handcrafted example files

* Profiles of differing number of "levels" (e.g. less than 36 bottles, ctds of different depth)
* variables that need more information (e.g. optical things which need wavelengths, pathlengths, angles, etc..)
* ACDD attrs at the variable level
* Combined ACDD variable level vars into global level (for compliance with actual ACDD-1.3)
* quality flags (following CF)
* compression? (zlib)
* Use of cchdo specific attrs
* "Complete" capture of "Bob's Header" in standardized attrs

Chosen cruise: 2016 I08S (33RR20160208)

In [2]:
# some flag defs...
water_sample = map(str.lower, [
    "Sample for this measurement was drawn from water bottle but analysis not received",
    "Acceptable measurement",
    "Questionable measurement",
    "Bad measurement",
    "Not reported",
    "Mean of replicate measurements",
    "Manual chromatographic peak measurement",
    "Irregular digital chromatographic peak integration",
    "Sample not drawn for this measurement from this bottle"
])
water_sample = map(str.split, water_sample)
water_sample = map("_".join, water_sample)
water_sample = " ".join(water_sample)

In [3]:
with open("33RR20160208_hy1.csv", 'r', encoding='utf8') as f:
    bottle_data = f.read()
comments = [line for line in bottle_data.splitlines() if line.startswith("#")]

def date_parser(dt):
    if dt == "nan nan":
        return None
    return pd.datetime.strptime(dt, "%Y%m%d        %H%M")

data = pd.read_csv("33RR20160208_hy1.csv", skiprows=[0,1], comment='#', header=[0], parse_dates=[["DATE", "TIME"]], date_parser=date_parser, na_values=[-999])
# drop the "units" and "END_DATA" for ease of working with
data = data.drop(0).drop(data.index[-1])

In [4]:
data["OXYGEN"] = data["OXYGEN"].astype(float)
data.loc[data["OXYGEN"]==-999, 'OXYGEN'] = np.nan
data["CTDSAL"] = data["CTDSAL"].astype(float)
data.loc[data["CTDSAL"]==-999, 'CTDSAL'] = np.nan
data["CTDTMP"] = data["CTDTMP"].astype(float)
data.loc[data["CTDTMP"]==-999, 'CTDTMP'] = np.nan
data["SALNTY"] = data["SALNTY"].astype(float)
data.loc[data["SALNTY"]==-999, 'SALNTY'] = np.nan

In [5]:
#non fancy select some stations
stn1 = data[:20]
stn2 = data[20:42]
stn6 = data[134:170]

stations = [stn1, stn2, stn6]

In [6]:
#lets grab some easy 1d vars
expocode = xr.DataArray(
    data=[s['EXPOCODE'].iloc[0] for s in stations],
    dims=['N_PROF'],
    attrs={
        "whp_name": "EXPOCODE"
    }
)
stnnbr = xr.DataArray(
    data=[str(int(s['STNNBR'].iloc[0])) for s in stations],
    dims=['N_PROF'],
    attrs={
        "whp_name": "STNNBR"
    }
)
castno = xr.DataArray(
    data=[int(s['CASTNO'].iloc[0]) for s in stations],
    dims=['N_PROF'],
    attrs={
        "whp_name": "CASTNO"
    },
    encoding={
        "dtype": np.int16
    }
)
woce = xr.DataArray(
    data=[str(s['SECT_ID'].str.strip().iloc[0]) for s in stations],
    dims=['N_PROF'],
    attrs={
        "whp_name": "SECT_ID"
    }
)
lat = xr.DataArray(
    data=[(s['LATITUDE'].iloc[0]) for s in stations],
    dims=['N_PROF'],
    attrs={
        "standard_name": "latitude",
        "units": "degrees_north",
        "axis": "Y",
    },
    encoding={
        "_FillValue": None,
        "dtype": np.float32
    }
)
lon = xr.DataArray(
    data=[(s['LONGITUDE'].iloc[0]) for s in stations],
    dims=['N_PROF'],
    attrs={
        "standard_name": "latitude",
        "units": "degrees_east",
        "axis": "X",
    },
    encoding={
        "_FillValue": None,
        "dtype": np.float32
    }
)
date = xr.DataArray(
    data=[(s['DATE_TIME'].iloc[0]) for s in stations],
    dims=['N_PROF'],
    attrs={
        "standard_name": "time",
        "axis": "T",
    },
    encoding={
        "_FillValue": None,
        "units": "minutes since 1970-01-01T00:00:00Z"
    }
)

In [7]:
# Lets try a 2d thing
ctdprs = xr.DataArray(
    data=pd.DataFrame([s['CTDPRS'].astype(float).values[::-1] for s in stations]).values,
    dims=['N_PROF', "N_LEVEL"],
    attrs={
        "standard_name": "sea_water_pressure",
        "units": "dbar",
        "axis": "Z",
        "positive": "down",
        "whp_name": "CTDPRS",
        "whp_unit": "DBAR"
    },
    encoding={
        "dtype": np.float32
    }
)
oxygen = xr.DataArray(
    data=pd.DataFrame([s['OXYGEN'].astype(float).values[::-1] for s in stations]).values,
    dims=['N_PROF', "N_LEVEL"],
    attrs={
        "standard_name": "moles_of_oxygen_per_unit_mass_in_sea_water",
        "units": "umol kg-1",
        "whp_name": "OXYGEN",
        "whp_unit": "UMOL/KG",
        "creator_name": "Swift, James",
        "creator_email": "jswift@ucsd.edu",
        "creator_institution": "Scripps Institution of Oceanography",
        "geospatial_bounds": "MULTIPOINT ((78.3815 -66.6027), (78.2986 -66.4997), (78.0102 -66.15))",
        "geospatial_lat_min": -66.6027,
        "geospatial_lat_max": -66.15,
        "geospatial_lon_min": 78.0102,
        "geospatial_lon_max": 78.3815,
        "creator_type": "person",
        "program": "GO-SHIP",
        "contributor_name": ["Barna, Andrew","Becker, Susan"],
        "contributor_role": ["Shipboard Tech","Shipboard Tech"],
        "geospatial_lat_units": "degrees_north",
        "geospatial_lon_units": "degrees_east",
        "date_issued": "2016-04-12",
        "processing_level": "final",
    },
    coords={"pressure": ctdprs, "latitude": lat, "longitude": lon, "date": date},
    encoding={
        "dtype": np.float32
    }
)
oxygen_qf = xr.DataArray(
    data=pd.DataFrame([s['OXYGEN_FLAG_W'].astype(int).values[::-1] for s in stations]).values,
    dims=['N_PROF', "N_LEVEL"],
    attrs={
        "standard_name": "status_flag",
        "whp_name": "OXYGEN_FLAG_W",
        "flag_values": np.array([1,2,3,4,5,6,7,8,9], dtype=np.int8),
        "flag_meanings": water_sample,
    },
    encoding={
        "dtype": np.int8,
        "_FillValue": 9,
    }
)

In [8]:
# oops... lets do the core ocean params
ctdtmp = xr.DataArray(
    data=pd.DataFrame([s['CTDTMP'].astype(float).values[::-1] for s in stations]).values,
    dims=['N_PROF', "N_LEVEL"],
    attrs={
        "standard_name": "sea_water_temperature",
        "units": "degc",
        "whp_name": "CTDTMP",
        "whp_unit": "ITS-90",
        "creator_name": "Swift, James",
        "creator_email": "jswift@ucsd.edu",
        "creator_institution": "Scripps Institution of Oceanography",
        "geospatial_bounds": "MULTIPOINT ((78.3815 -66.6027), (78.2986 -66.4997), (78.0102 -66.15))",
        "geospatial_lat_min": -66.6027,
        "geospatial_lat_max": -66.15,
        "geospatial_lon_min": 78.0102,
        "geospatial_lon_max": 78.3815,
        "creator_type": "person",
        "program": "GO-SHIP",
        "contributor_name": ["Joseph Gum","Becker, Susan"],
        "contributor_role": ["Shipboard Tech","Shipboard Tech"],
        "geospatial_lat_units": "degrees_north",
        "geospatial_lon_units": "degrees_east",
        "date_issued": "2016-04-12",
        "processing_level": "final",
        "reference_scale": "ITS-90" # taken from oceansites
    },
    coords={"pressure": ctdprs, "latitude": lat, "longitude": lon, "date": date},
    encoding={
        "dtype": np.float32
    }
)
# NO QF for ctdtmp
ctdsal = xr.DataArray(
    data=pd.DataFrame([s['CTDSAL'].astype(float).values[::-1] for s in stations]).values,
    dims=['N_PROF', "N_LEVEL"],
    attrs={
        "standard_name": "sea_water_practical_salinity",
        "units": "1",
        "whp_name": "CTDSAL",
        "whp_unit": "PSS-78",
        "creator_name": "Swift, James",
        "creator_email": "jswift@ucsd.edu",
        "creator_institution": "Scripps Institution of Oceanography",
        "geospatial_bounds": "MULTIPOINT ((78.3815 -66.6027), (78.2986 -66.4997), (78.0102 -66.15))",
        "geospatial_lat_min": -66.6027,
        "geospatial_lat_max": -66.15,
        "geospatial_lon_min": 78.0102,
        "geospatial_lon_max": 78.3815,
        "creator_type": "person",
        "program": "GO-SHIP",
        "contributor_name": ["Joseph Gum","Becker, Susan"],
        "contributor_role": ["Shipboard Tech","Shipboard Tech"],
        "geospatial_lat_units": "degrees_north",
        "geospatial_lon_units": "degrees_east",
        "date_issued": "2016-04-12",
        "processing_level": "final",
        "instrument": "SeaBird SBE 4"
    },
    coords={"pressure": ctdprs, "latitude": lat, "longitude": lon, "date": date},
    encoding={
        "dtype": np.float32
    }
)
ctdsal_qf = xr.DataArray(
    data=pd.DataFrame([s['CTDSAL_FLAG_W'].astype(int).values[::-1] for s in stations]).values,
    dims=['N_PROF', "N_LEVEL"],
    attrs={
        "standard_name": "status_flag",
        "whp_name": "CTDSAL_FLAG_W",
        "flag_values": np.array([1,2,3,4,5,6,7,8,9], dtype=np.int8),
        "flag_meanings": water_sample,
    },
    encoding={
        "dtype": np.int8,
        "_FillValue": 9,
    }
)
salnty = xr.DataArray(
    data=pd.DataFrame([s['SALNTY'].astype(float).values[::-1] for s in stations]).values,
    dims=['N_PROF', "N_LEVEL"],
    attrs={
        "standard_name": "sea_water_practical_salinity",
        "units": "1",
        "whp_name": "CTDSAL",
        "whp_unit": "PSS-78",
        "creator_name": "Swift, James",
        "creator_email": "jswift@ucsd.edu",
        "creator_institution": "Scripps Institution of Oceanography",
        "geospatial_bounds": "MULTIPOINT ((78.3815 -66.6027), (78.2986 -66.4997), (78.0102 -66.15))",
        "geospatial_lat_min": -66.6027,
        "geospatial_lat_max": -66.15,
        "geospatial_lon_min": 78.0102,
        "geospatial_lon_max": 78.3815,
        "creator_type": "person",
        "program": "GO-SHIP",
        "contributor_name": ["John Calderwood","Hembrough, Bret"],
        "contributor_role": ["Shipboard Tech","Shipboard Tech"],
        "geospatial_lat_units": "degrees_north",
        "geospatial_lon_units": "degrees_east",
        "date_issued": "2016-04-12",
        "processing_level": "final",
        "instrument": "Guildline 8400B"
    },
    coords={"pressure": ctdprs, "latitude": lat, "longitude": lon, "date": date},
    encoding={
        "dtype": np.float32
    }
)
salnty_qf = xr.DataArray(
    data=pd.DataFrame([s['SALNTY_FLAG_W'].astype(int).values[::-1] for s in stations]).values,
    dims=['N_PROF', "N_LEVEL"],
    attrs={
        "standard_name": "status_flag",
        "whp_name": "SALNTY_FLAG_W",
        "flag_values": np.array([1,2,3,4,5,6,7,8,9], dtype=np.int8),
        "flag_meanings": water_sample,
    },
    encoding={
        "dtype": np.int8,
        "_FillValue": 9,
    }
)

In [9]:
# pick another param, remove a station for demo purposes
ph_data = pd.DataFrame([s['PH_TOT'].astype(float).values[::-1] for s in stations]).values
ph_data[0] = np.nan
ph_qf_data = pd.DataFrame([s['PH_TOT_FLAG_W'].astype(int).values[::-1] for s in stations]).values
ph_qf_data[0] = np.nan
ph = xr.DataArray(
    data=ph_data,
    dims=['N_PROF', "N_LEVEL"],
    attrs={
        "standard_name": "sea_water_ph_reported_on_total_scale",
        "units": "1",
        "whp_name": "PH_TOT",
        "creator_name": "Dickson, Andrew",
        "creator_email": "adickson@ucsd.edu",
        "creator_institution": "Scripps Institution of Oceanography",
        "geospatial_bounds": "MULTIPOINT ((78.2986 -66.4997), (78.0102 -66.15))",
        "geospatial_lat_min":-66.4997,
        "geospatial_lat_max": -66.15,
        "geospatial_lon_min": 78.0102,
        "geospatial_lon_max": 78.2986,
        "creator_type": "person",
        "program": "GO-SHIP",
        "contributor_name": ["Belmonte, Manuel","Cervantes, David"],
        "contributor_role": ["Shipboard Tech","Shipboard Tech"],
        "geospatial_lat_units": "degrees_north",
        "geospatial_lon_units": "degrees_east",
        "date_issued": "2016-04-12",
        "processing_level": "final",
    },
    coords={"pressure": ctdprs, "latitude": lat, "longitude": lon, "date": date},
    encoding={
        "dtype": np.float32
    }
)
ph_qf = xr.DataArray(
    data=ph_qf_data,
    dims=['N_PROF', "N_LEVEL"],
    attrs={
        "standard_name": "status_flag",
        "whp_name": "PH_TOT_FLAG_W",
        "flag_values": np.array([1,2,3,4,5,6,7,8,9], dtype=np.int8),
        "flag_meanings": water_sample,
    },
    encoding={
        "dtype": np.int8,
        "_FillValue": 9,
    }
)

In [10]:
# some "fake CDOM data"
fake = pd.read_csv("cdom.csv", header=None, na_values=[-999, 9]).values
wavelength = xr.DataArray(
    data=[325, 370, 443],
    dims=["N_WAVELENGTH0"],
    attrs={
        "standard_name": "radiation_wavelength",
        "units": "nm"
    },
    encoding={
        "dtype": np.int32
    }
)
cdom_data = np.empty((3,36,3))
cdom_data.fill(np.nan)
cdom_data[2,:,0] = fake[:,0]
cdom_data[2,:,1] = fake[:,2]
cdom_data[2,:,2] = fake[:,4]
cdom_qc = np.empty((3,36,3))
cdom_qc.fill(np.nan)
cdom_qc[2,:,0] = fake[:,1]
cdom_qc[2,:,1] = fake[:,3]
cdom_qc[2,:,2] = fake[:,5]
cdom = xr.DataArray(
    data=cdom_data,
    dims=['N_PROF', "N_LEVEL", "N_WAVELENGTH0"],
    coords={"pressure": ctdprs, "latitude": lat, "longitude": lon, "date": date, "wavelength0":wavelength},
    attrs={
        "standard_name": "volume_beam_attenuation_coefficient_of_radiative_flux_in_sea_water_due_to_colored_dissolved_organic_matter", #this name isn't real
        "units": "m-1",
        "whp_name": "CDOM{wavelength0}",
        "whp_unit": "1/M",
        "creator_name": "笹岡 晃征 (Kosei Sasaoka)",
        #"creator_name": "Kosei Sasaoka",
        "creator_email": "sasaoka@jamstec.go.jp",
        "creator_institution": "Japan Agency for Marine-Earth Science and Technology",
        "creator_type": "person",
        "geospatial_bounds": "MULTIPOINT ((78.0102 -66.15))",
        "geospatial_lat_min":-66.15,
        "geospatial_lat_max": -66.15,
        "geospatial_lon_min": 78.0102,
        "geospatial_lon_max": 78.0102,
        "program": "GO-SHIP",
        "geospatial_lat_units": "degrees_north",
        "geospatial_lon_units": "degrees_east",
        "date_issued": "2016-04-12",
        "processing_level": "preliminary"
    },
    encoding={
        "dtype": np.float32
    }
)
cdom_qf = xr.DataArray(
    data=cdom_qc,
    dims=['N_PROF', "N_LEVEL", "N_WAVELENGTH0"],
    coords={"pressure": ctdprs, "latitude": lat, "longitude": lon, "date": date, "wavelength0":wavelength}, 
    attrs={
        "standard_name": "status_flag",
        "whp_name": "CDOM{wavelength0}_FLAG_W",
        "flag_values": np.array([1,2,3,4,5,6,7,8,9], dtype=np.int8),
        "flag_meanings": "def1 def2 def3 def4 def5 def6 def7 def8 def9",
    },
    encoding={
        "dtype": np.int8,
        "_FillValue": 9,
    }
)

In [11]:
example = xr.Dataset(data_vars={
    "var0": oxygen, 
    "var1": oxygen_qf, 
    "var2": ph, 
    "var3": ph_qf, 
    "var4": cdom,
    "var5": cdom_qf,
    "var10": ctdtmp,
    "var11": ctdsal,
    "var12": ctdsal_qf,
    "var13": salnty,
    "var14": salnty_qf,
    },
    coords={
        "wavelength0": wavelength,
        "var6": expocode,
        "var7": stnnbr, 
        "var8": castno, 
        "var9": woce,
    },
    attrs={
        "Conventions": "CF-1.7 ACDD-1.3 CCHDO-0.0",
        "title": "CCHDO netCDF example file",
        "warning": "The data are not real, this is only a file to demo features of the work in progress"
    }
)

In [12]:
example.var0.attrs["ancillary_variables"] = "var1"
example.var2.attrs["ancillary_variables"] = "var3"
example.var4.attrs["ancillary_variables"] = "var5"
example.var11.attrs["ancillary_variables"] = "var12"
example.var13.attrs["ancillary_variables"] = "var14"

In [13]:
from collections import defaultdict
from dataclasses import dataclass
from itertools import starmap

combining = (
    "creator_name",
    "creator_email",
    "creator_institution",
    "geospatial_bounds",
    "geospatial_lat_min",
    "geospatial_lat_max",
    "geospatial_lon_min",
    "geospatial_lon_max",
    "creator_type",
    "program",
    "contributor_name",
    "contributor_role",
    "geospatial_lat_units",
    "geospatial_lon_units",
    "date_issued",
)


def combine_creators(attrd):
    @dataclass(frozen=True)
    class Creator:
        name: str
        email: str
        institution: str
        ctype: str # type is a reserved word

    combined = {
        "creator_name":[],
        "creator_email":[],
        "creator_institution":[],
        "creator_type": []
    }
                
    for key in combined:
        for _, value in attrd[key].items():
            if type(value) == list:
                combined[key].extend(value)
            else:
                combined[key].append(value)
    creators = {*starmap(Creator, (zip(*combined.values())))} # create a set of Creator objects using magic, note that order may be "hash" dependent
    return {
        "creator_name": ",".join([f"\"{c.name}\"" for c in creators]),
        "creator_email": ",".join([f"\"{c.email}\"" for c in creators]),
        "creator_institution": ",".join([f"\"{c.institution}\"" for c in creators]),
        "creator_type": ",".join([f"\"{c.ctype}\"" for c in creators])
    }
def combine_contributor(attrd):
    @dataclass(frozen=True)
    class Contributor:
        name: str
        role: str

    combined = {
        "contributor_name":[],
        "contributor_role":[],
    }
                
    for key in combined:
        for _, value in attrd[key].items():
            if type(value) == list:
                combined[key].extend(value)
            else:
                combined[key].append(value)
    creators = {*starmap(Contributor, (zip(*combined.values())))} # create a set of Creator objects using magic, note that order may be "hash" dependent
    return {
        "contributor_name": ",".join([f"\"{c.name}\"" for c in creators]),
        "contributor_role": ",".join([f"\"{c.role}\"" for c in creators]),
    }

attrs = defaultdict(dict)
for var in example:
    for key, value in example[var].attrs.items():
        attrs[key][var] = value
        
example.attrs.update(combine_creators(attrs))
example.attrs.update(combine_contributor(attrs))
example.attrs.update({
    "geospatial_bounds": "MULTIPOINT ((78.3815 -66.6027), (78.2986 -66.4997), (78.0102 -66.15))",
    "geospatial_lat_min": -66.6027,
    "geospatial_lat_max": -66.15,
    "geospatial_lon_min": 78.0102,
    "geospatial_lon_max": 78.3815,
    "program": "GO-SHIP",
    "geospatial_lat_units": "degrees_north",
    "geospatial_lon_units": "degrees_east",
    "date_issued": "2016-04-12",
    "publisher_type": "group",
    "publisher_institution": "Scripps Institution of Oceanography",
    "publisher_name": "CCHDO",
    "publisher_email": "cchdo@ucsd.edu"
})
example.encoding["_Encoding"]= "utf8"

In [14]:
from collections import OrderedDict
def sortify_attrs(xr_ds):
    for var in xr_ds:
        xr_ds[var].attrs = OrderedDict(sorted(xr_ds[var].attrs.items(), key=lambda item: item[0]))
    xr_ds.attrs = OrderedDict(sorted(xr_ds.attrs.items(), key=lambda item: item[0]))
sortify_attrs(example)


In [15]:
example.to_netcdf("example_cchdo_utf8.nc")